In [14]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import core
import keras
import core.keras_models.RegressionTransformer as RegressionTransformer

PLOTS_DIR = f"plots/regression_transformer/"
MODEL_DIR = f"models/regression_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], max_events=4_000_000, event_numbers="even"
)

X_train, y_train = DataProcessor.get_data()
del DataProcessor  # Free memory

In [ ]:
reload(RegressionTransformer)
Transformer = RegressionTransformer.FeatureConcatTransformerReconstructor(data_config, name="Transformer")

In [20]:
Transformer.build_model(
    hidden_dim=64,
    num_layers=6,
    dropout_rate=0.1,
)


In [24]:
Transformer.adapt_normalization_layers(X_train)
Transformer.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(),
        "normalized_regression": core.utils.MagnitudeDirectionLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy(name="assignment_accuracy")],
        "normalized_regression": [core.utils.RegressionDeviation()],
    },
    #add_physics_informed_loss=True,
    loss_weights={"assignment": 1.0, "normalized_regression": 1.0}
)

Adapted normalization layer:  hlf_input_normalization
Adapted normalization layer:  met_input_normalization
Adapted normalization layer:  lep_input_normalization
Adapted normalization layer:  jet_input_normalization


In [ ]:
Transformer.train_model(
    epochs=10,
    data=X_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
    validation_split=0.1,
)

Epoch 1/10
3511/3512 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - assignment_assignment_accuracy: 0.5026 - assignment_loss: 0.1422 - loss: 2.0301 - normalized_regression_deviation: 0.5387 - normalized_regression_loss: 1.8879

2026-02-03 17:37:27.595810: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_10', 124 bytes spill stores, 104 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_12', 104 bytes spill stores, 88 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_13', 104 bytes spill stores, 88 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_2', 16 bytes spill stores, 16 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_multiply_reduce_fusion_9', 16 bytes spill stores, 16 bytes spill loads



3512/3512 ━━━━━━━━━━━━━━━━━━━━ 346s 78ms/step - assignment_assignment_accuracy: 0.5026 - assignment_loss: 0.1422 - loss: 2.0301 - normalized_regression_deviation: 0.5387 - normalized_regression_loss: 1.8879 - val_assignment_assignment_accuracy: 0.5505 - val_assignment_loss: 0.1366 - val_loss: 1.9695 - val_normalized_regression_deviation: 0.5370 - val_normalized_regression_loss: 1.8329 - learning_rate: 1.0000e-04
Epoch 2/10
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 232s 66ms/step - assignment_assignment_accuracy: 0.5454 - assignment_loss: 0.1367 - loss: 2.0113 - normalized_regression_deviation: 0.5362 - normalized_regression_loss: 1.8745 - val_assignment_assignment_accuracy: 0.5845 - val_assignment_loss: 0.1323 - val_loss: 1.9511 - val_normalized_regression_deviation: 0.5146 - val_normalized_regression_loss: 1.8187 - learning_rate: 1.0000e-04
Epoch 3/10
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 232s 66ms/step - assignment_assignment_accuracy: 0.5755 - assignment_loss: 0.1333 - loss: 1.9928 - normalized_regre

In [ ]:
Transformer.save_model(MODEL_DIR + "odd_model.keras")

In [ ]:
upscale_layer = Transformer.model.get_layer("regression")

In [ ]:
import numpy as np
import tensorflow as tf
np.abs(upscale_layer(y_train["regression"] / y_train["regression"].std(axis = 0)) - X_train["regression"]).mean()

In [ ]:
from core.components import reco_W_mass_deviation
leptons = X_train["lep_inputs"]
neutrino_pred = Transformer.reconstruct_neutrinos(X_train)
neutrino_true = X_train["regression"]
neutrino_nu_flows = X_train["nu_flows_neutrino_truth"]

In [ ]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_pred, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()

In [ ]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_true, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()

In [ ]:
reco_W_mass_deviation(tf.convert_to_tensor(neutrino_nu_flows, dtype=tf.float32), tf.convert_to_tensor(leptons, dtype=tf.float32)).numpy().mean()